In [ ]:
# conda install -c conda-forge openexr

In [ ]:
import os
import ntpath
import re

import math

import multiprocessing

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras

#import runai.ga

#from nvidia.dali.pipeline import Pipeline
#import nvidia.dali.ops as ops
#import nvidia.dali.types as types

from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils.np_utils import to_categorical

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from imgaug import augmenters as iaa

import tensorflow as tf
from tensorflow.python.client import device_lib

import datetime

import cv2
import pandas as pd
import random
from IPython.display import Image
from pathlib import Path

import glob
import json
#import OpenEXR
import imageio

In [ ]:
print( "Number of processors: ", multiprocessing.cpu_count() )

In [ ]:
# set to true to get more output for debugging issues
print_debug = False

# path to folder with all json files with object information
path_to_images = "C:\\tmp\\dump_truck4"
#path_to_images = "C:\\tmp\\test"


included_extensions = [ 'json' ] #, 'jpg','jpeg', 'bmp', 'png', 'gif']
obj_data_files = [ fn for fn in os.listdir( path_to_images )
                      if any( fn.endswith( ext ) for ext in included_extensions ) ]
print( "Found ", len( obj_data_files ), " json files" )

In [ ]:
def load_obj_data( jsonfile ):
    json_path = os.path.join( path_to_images, jsonfile )
    with open( json_path, 'r' ) as infile:
        data = json.load( infile )
    return data

In [ ]:
def get_file_name(json_data):
    img_files_pre = json_data["frame"]["files"]
    img_files = [os.path.normpath(f) for f in img_files_pre]
    path_img           = [f for f in img_files if re.match(r'.*_img[0-9]+\.(png|jpeg|jpg)', f)][0]
    path_img_z_map     = [f for f in img_files if re.match(r'.*_z[0-9]+\.exr', f)][0]
    path_img_class     = [f for f in img_files if re.match(r'.*_class[0-9]+\.(png|jpeg|jpg)', f)][0]
    path_img_z_img     = [f for f in img_files if re.match(r'.*_z_img[0-9]+\.(png|jpeg|jpg)', f)][0]
    path_img_class_img = [f for f in img_files if re.match(r'.*_class_img[0-9]+\.(png|jpeg|jpg)', f)][0]
    return (path_img, path_img_z_map, path_img_class, path_img_z_img, path_img_class_img )

In [ ]:
def img_preprocess_load( img_path ):
    img = mpimg.imread( img_path )
    img = img_preprocess( img )
    return img

In [ ]:
def img_rm_alpha( img_orig ):
    img = np.copy( img_orig )
    h, w, cp = img.shape
    if cp == 4:
        for y in range( h - 1 ):
            for x in range( w - 1 ):
                img[y][x][3]=1
    return img

In [ ]:
def img_rect( img_orig, rects ):
    img = img_rm_alpha( img_orig )
    h, w, cp = img.shape
    print ("H,W,CP = ", h, w, cp)
    for rect in rects:
        (p1, p2, color) = rect
        p1s = ( int(p1[0] * w), int(h - p1[1] * h) )
        p2s = ( int(p2[0] * w), int(h - p2[1] * h) )
        cv2.rectangle( img, p1s, p2s, color, 5 )
    alpha = 0.4  # Transparency factor.
    # Following line overlays transparent rectangle over the image
    img = cv2.addWeighted(img, alpha, img_orig, 1 - alpha, 0)
    return img

def rndColor():
    r = min(random.randint(0, 8) * 32, 255)
    g = min(random.randint(0, 8) * 32, 255)
    b = min(random.randint(0, 8) * 32, 255)
    return (r, g, b)

def get_bb(data, objName):
    c = rndColor()
    if print_debug:
        print ("Color for ", objName, " is ", c)

    objData = data["frame"]["objects"][objName]
    if "bb2dWithChildren" in objData:
        bb2d = objData["bb2dWithChildren"]
    elif "bb2d" in objData:
        bb2d = objData["bb2d"]
    if bb2d:
        rect = ( ( bb2d["x1"], bb2d["y1"] ), ( bb2d["x2"], bb2d["y2"] ), c )
    else:
        rect = ( ( 0, 0 ), ( 0, 0 ) )
    return rect

In [ ]:
def output_bounding_box(values):
    # output debugging information about the bounding boxes
    if "bb2d" in values:
        print("{}\t{}\tbb2d=({:.2},{:.2},{:.2},{:.2})".format(keys,values["name"],values["bb2d"]["x1"],values["bb2d"]["y1"],values["bb2d"]["x2"],values["bb2d"]["y2"]) )
    if "bb2dWithChildren" in values:
        print("{}\t{}\tbb2dWC=({:.2},{:.2},{:.2},{:.2})".format(keys,values["name"],values["bb2dWithChildren"]["x1"],values["bb2dWithChildren"]["y1"],values["bb2dWithChildren"]["x2"],values["bb2dWithChildren"]["y2"]) )

In [ ]:
print_debug = False

def loadFileInfo( obj_data_path ):
    #objs = [ "Cube.000", "Cube.002", "Cube.003" ]
    #objs = [ "Car Rig", "wheel.Ft.L", "wheel.Ft.R", "wheel.Bk.L", "wheel.Bk.R", "body", "Minetruck" ]
    #objs = [ "wheel.Ft.L", "wheel.Ft.R", "wheel.Bk.L", "wheel.Bk.R", "body", "Minetruck" ]
    #objs = [ "testcar-Body", "testcar-Wheel.Ft.L", "testcar-Wheel.Ft.R", "testcar-Wheel.Bk.L", "testcar-Wheel.Bk.R", "Car Rig" ]
    objs = [ "Car Rig", "wheel.Ft.L", "wheel.Ft.R", "wheel.Bk.L", "wheel.Bk.R" ]

    # load json information
    data = load_obj_data( obj_data_path )

    # load image for this description
    path_img, path_img_z_map, path_img_class, path_img_z_img, path_img_class_img = get_file_name( data )
    #if print_debug:
    print ( "Loading ", obj_data_path, " using image file ", path_img )
    img_original = mpimg.imread( path_img )

    if print_debug:
        # output debugging information about the bounding boxes
        for keys, values in data[ "frame" ][ "objects" ].items():
            output_bounding_box( values )

    # find information about the steering of the vehicle
    for keys, values in data[ "frame" ][ "objects" ][ objs[ 0 ] ][ "pose" ][ "bones" ].items():
        if values[ "name" ] == "Steering":
            steering_bone = values
        elif values[ "name" ] == "GroundSensor.Axle.Ft":
            front_axle_bone = values

    if print_debug and steering_bone:
        print( "Found steering bone: ", steering_bone[ "head" ] )
    if print_debug and front_axle_bone:
        print( "Found front axle bone: ", front_axle_bone[ "head" ] )
    steering_angle = 0.0
    if steering_bone[ "head" ][ "y" ] != 0.0:
        diff_x = steering_bone[ "head" ][ "x" ] - front_axle_bone[ "head" ][ "x" ]
        diff_y = steering_bone[ "head" ][ "y" ] - front_axle_bone[ "head" ][ "y" ]
        steering_angle = 180.0 / math.pi * math.atan( diff_x / diff_y )
    if print_debug:
        print( "steering_angle = ", steering_angle, " degree" )

    rects = [ get_bb( data, o ) for o in objs ]
    return ( steering_angle, rects, img_original )

In [ ]:
# run single-threaded
objectInfos = [ loadFileInfo( n ) for n in obj_data_files ]

# run multi-threaded:
#with mp.Pool( processes = mp.cpu_count() ) as pool:
#    objectInfos = pool.map( loadFileInfo, [ n for n in obj_data_files[ :20 ] ] )

In [ ]:
# draw the plots + bounding boxes
fig, axs = plt.subplots( 4, 2, figsize = ( 15, 30 ) )
fig.tight_layout()
for y in range(4):
    for x in range(2):
        obj_data_idx = random.randint( 0, len( objectInfos ) - 1 )
        ( steering_angle, bbs, img ) = objectInfos[ obj_data_idx ]
        print ( "steering angle = ", steering_angle, " for image ", obj_data_idx )
        axs[ y, x ].imshow( img_rect( img, bbs ) )
        axs[ y, x ].grid( color='w', linestyle='-', linewidth = 1 )
        axs[ y, x ].set_title( 'Original Image' )

In [ ]:
image_extensions = [ 'jpg','jpeg', 'bmp', 'png', 'gif']
background_image_source_folder = "S:\\Bilder_Filme"

def get_sub_dirs( p ):
    subdirs = [ p ]
    for x in p.iterdir():
        if x.is_dir():
            subdirs = subdirs + get_sub_dirs( x )
    return subdirs


bgimg_path = Path( background_image_source_folder )
subdirs = get_sub_dirs( Path( bgimg_path ) )
img_files = []
for path in subdirs:
    print ( "Checking ", path )
    img_files = img_files + [ fn for fn in os.listdir( path ) if any( fn.endswith( ext ) for ext in image_extensions ) ]

print( "Found ", len( img_files ), " images files" )
print( "Images: ", img_files )

In [ ]:
# img_files
# * load some images
# * write function to add random background image to a given object image detection
# * testing
# * ... (move forward to AI training)